In [1]:
import numpy as np
from tensorflow import keras

import matplotlib.pyplot as plt
import os
import cv2
import random
import sklearn.model_selection as model_selection
import datetime
from model import createModel
from contextlib import redirect_stdout
from tensorflow.keras import layers
import tensorflow as tf

# The following model and it's predictions should not be considered as  a valid medical advice

In [3]:
categories = ["NonDemented", "MildDemented", "ModerateDemented", "VeryMildDemented"]
SIZE = 120


In [4]:
def getData():
    rawdata = []
    data = []
    dir = "./data/"
    for category in categories:
        path = os.path.join(dir, category)
        class_num = categories.index(category)
        for img in os.listdir(path):
            try:
                rawdata = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                new_data = cv2.resize(rawdata, (SIZE, SIZE))

                data.append([new_data, class_num])
            except Exception as e:
                pass

    random.shuffle(data)

    img_data = []
    img_labels = []
    for features, label in data:
        img_data.append(features)
        img_labels.append(label)
    img_data = np.array(img_data).reshape(-1, SIZE, SIZE, 1)
    img_data = img_data / 255.0
    img_labels = np.array(img_labels)

    return img_data, img_labels


In [5]:
data, labels = getData()


In [6]:
labels

array([3, 3, 3, ..., 3, 0, 0])

In [7]:
train_data, test_data, train_labels, test_labels = model_selection.train_test_split(data, labels, test_size=0.20)

train_data, val_data, train_labels, val_labels = model_selection.train_test_split(train_data, train_labels,
                                                                                  test_size=0.10)

In [8]:
 model = keras.Sequential([

            keras.Input(shape=train_data.shape[1:]),
            layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Flatten(),
            layers.Dense(128, activation="relu"),

            layers.Dropout(0.5),
            layers.Dense(4, activation="softmax")

        ])

In [9]:
checkpoint = keras.callbacks.ModelCheckpoint(filepath='./model/model.h5', save_best_only=True, monitor='val_loss',
                                             mode='min')


                    

In [10]:
opt = keras.optimizers.Adam(learning_rate=0.001)


In [11]:
model.compile(optimizer=opt, loss="sparse_categorical_crossentropy", metrics=["accuracy"], )

  opt = tf.keras.mixed_precision.experimental.LossScaleOptimizer(opt)



In [12]:
history = model.fit(train_data, train_labels, epochs=10, validation_data=(val_data, val_labels))

Epoch 1/10
144/144 [==============================] - 8s 41ms/step - loss: 1.0998 - accuracy: 0.4602 - val_loss: 1.0139 - val_accuracy: 0.4805
Epoch 2/10
144/144 [==============================] - 5s 38ms/step - loss: 0.9223 - accuracy: 0.5639 - val_loss: 0.8175 - val_accuracy: 0.6172
Epoch 3/10
144/144 [==============================] - 5s 38ms/step - loss: 0.7825 - accuracy: 0.6453 - val_loss: 0.6765 - val_accuracy: 0.6914
Epoch 4/10
144/144 [==============================] - 5s 38ms/step - loss: 0.6166 - accuracy: 0.7284 - val_loss: 0.5143 - val_accuracy: 0.7891
Epoch 5/10
144/144 [==============================] - 5s 38ms/step - loss: 0.4597 - accuracy: 0.8018 - val_loss: 0.4241 - val_accuracy: 0.8379
Epoch 6/10
144/144 [==============================] - 5s 38ms/step - loss: 0.3703 - accuracy: 0.8480 - val_loss: 0.3468 - val_accuracy: 0.8750
Epoch 7/10
144/144 [==============================] - 5s 38ms/step - loss: 0.2519 - accuracy: 0.8974 - val_loss: 0.2530 - val_accuracy: 0.9062